# tensorflow基础

TensorFlow主要依赖包
ProtocolBuffer（结构化信息工具）
Bazel自动化构建工具

In [1]:
import tensorflow as tf

除了使用默认的计算图外，还可以生成新的计算图，张量和运算不会共享

In [11]:
g1 = tf.Graph()
with g1.as_default():
#    v = tf.get_variable("v", initializer=tf.zeros_initializer(shape=[1]))
#新版上述写法会报错
    v = tf.get_variable("v", shape=[1], initializer=tf.zeros_initializer())

with tf.Session(graph = g1) as sess:
#    tf.initialize_all_variables().run()
#新版上述写法会被移除
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse = True):
        print(sess.run(tf.get_variable("v")))

[0.]


还可以指定计算图运行的设备

In [ ]:
g = tf.Graph()
with g.device("/gpu:0"):
    result = a + b
##没有GPU就不运行了

tensorflow计算结果是一个张量

tensorflow支持14种变量类型，包括tf.float32/tf.float64/tf.int8/tf.in16/tf.int32/tf.bool/tf.complex64（复数）等等

# tensorflow两种运行模型

In [ ]:
sess = tf.Session()
sess.run(...)
sess.close()

In [ ]:
with tf.Session() as sess:
    sess.run(...)

以下命令效果相同

In [ ]:
sess.run(result)
#=
result.eval(session=sess)

交互式环境

In [ ]:
sess = tf.InteractiveSession()
result.eval()
sess.close()

配置对话需要用configproto protocol buffer

In [12]:
config = tf.ConfigProto(allow_soft_placement = True,#是否可以在CPU上运行
                       log_device_placement = True)#是否记录节点所在设备
sess1 = tf.Session(config = config)
sess2 = tf.InteractiveSession(config = config)

# tensorflow变量

In [15]:
weights = tf.Variable(tf.random_normal([2,3],stddev=2))
#初始化2*3矩阵随机变量

几种随机数生成函数 

tf.random_normal 正态分布

tf.truncated_normal 两个标准差以内的正态分布

tf.random_uniform 均匀分布

tf.random_gamma 随机Gamma分布

几种常数初始化

tf.zeros

tf.ones

tf.fill 产生全为一个值的常量

tf.constant 产生给定值常量

In [16]:
#也可以通过其他变量的初始值来初始化新的变量
w2 = tf.Variable(weights.initialized_value()*2)

变量初始化的两种方法

In [17]:
sess = tf.Session()
sess.run(w2.initializer)
#============================
tf.global_variables_initializer().run()
sess.close()

将一个变量赋值给另一个变量

要求类型一致，默认validate_shape=True要求长度一致

In [ ]:
w1.assign(w2)
#===========
tf.assign(w1, w2, validate_shape=False)

# TensorFlow参数训练

Tensorflow的Placeholder

In [19]:
x = tf.placeholder(tf.float32, shape=(3,2),name="input")
#sess.run(y, feed_dict{x:...})

损失函数与反向传播算法

In [ ]:
#损失函数 交叉熵 -\sum p(x)log q(x)
cross_entropy=-tf.reduce_mean(
    y_*tf.log(tf.clip_by_value(y,1e-10,1.0)))
#学习率
learning_rate = 0.001
#反向传播算法
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

三种优化方法 GradientDescentOptimizer AdamOptimizer MomentumOptimizer

矩阵乘法matmul

深层网络：去线性化relu

In [ ]:
a = tf.nn.relu(tf.matmul(x, w1) + biases1)

深层网络：损失函数softmax e^{x_i}/\sum e^{x_j}

In [ ]:
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, y_)

In [ ]:
#均方误差
mse = tf.reduce_mean(tf.square(y_-y))

两个tf函数

tf.select 

tf.greater

学习率随迭代逐步减小

tf.train.exponential_decay

In [23]:
global_step = tf.Variable(0)
learning_rate = tf.train.exponential_decay(
    0.1, global_step, 100, 0.96, staircase = True)
#初始0.1，每100次乘以0.96

防止过拟合：正则化参数(L2正则化）

In [ ]:
lambda1 = 1
tf.contrib.layers.l2_regularizer(lambda1)(w)

使用集合的方法提高代码可读性

In [ ]:
tf.add_to_collection("losses",...)
tf.add_to_collection("losses",...)
#然后获取总和
tf.add_n(tf.get_collection("losses"))

更加稳健：滑动平均模型

In [ ]:
step = tf.Variable(0, trainable=False)#训练轮数
#定义一个滑动平均类
ema = tf.train.ExponentialMovingAverage(0.99, step)
#定义一个列表 每次更新列表中的变量
maintain_averages_op = ema.apply([v1])
#更新变量
sess.run(maintain_averages_op)
#计算滑动平均结果
ema.average(v1)

# TensorFlow 变量管理

变量定义的两种方式

In [30]:
v1 = tf.get_variable("v1",shape=[1],
                   initializer=tf.constant_initializer(1.0))
#============
v2 = tf.Variable(tf.constant(1.0, shape=[1]), name = "v2")

七种初始化函数

tf.constant_initializer

tf.random_normal_initializer

tf.truncated_normal_initializer

tf.random_uniform_initializer

tf.random_unif_scaling_initializer

tf.zeros_initializer

tf.ones_initializer

同一个命名空间重复使用同一个名字会报错，解决方法在命名空间加入reuse=True

In [31]:
with tf.variable_scope("foo", reuse=True):
    v=tf.get_variable("v",[1])
#实际变量名为“foo/v”

# 模型的保存

In [1]:
import tensorflow as tf
v1 = tf.Variable(tf.constant(1,shape=[1]), name = "v1")
v2 = tf.Variable(tf.constant(2,shape=[1]), name = "v2")
result = v1+v2
saver=tf.train.Saver()

In [2]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.save(sess,"model/model_example.ckpt")

save的参数global_step=1000文件名会命名为model.ckpt-1000

会出现三个文件，.meta保存计算图结构，.ckpt保存变量取值，checkpoint文件保存目录下所有模型文件列表

加载模型使用保存模型一样的方式声明变量

In [11]:

v1 = tf.Variable(tf.constant(1,shape=[1]), name = "v1")
v2 = tf.Variable(tf.constant(2,shape=[1]), name = "v2")
result = v1+v2
#如果不加入下一行，在同一个文件运行会报错 
#（因为变量名相同的变量不能重复定义）
saver=tf.train.Saver({"v1":v1,"v2":v2})
with tf.Session() as sess:
    saver.restore(sess,"model/model_example.ckpt")
    print (sess.run(result))

INFO:tensorflow:Restoring parameters from model/model_example.ckpt
[3]


第二种加载的方式 直接加载图

In [4]:
saver = tf.train.import_meta_graph("model/model_example.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess,"model/model_example.ckpt")
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))

INFO:tensorflow:Restoring parameters from model/model_example.ckpt
[3]


注意tf.train.Saver()默认加载了所有变量，如果tf.train.Saver([v1])只会加载v1

可以通过定义重命名变量字典的方式加载模型，比如
tf.train.Saver({"v1":v1}) "V1"指的是存储的图中变量，v1是新的变量

在滑动平均类下提供variable_to_restore()的方法，相当于上述字典

In [ ]:
#可以将变量保存为常量 
from tensorflow.python.framework import graph_util
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph_def=
    graph_util.convert_variables_to_constants(sess, graph_def, ["add"])
    with tf.gfile.GFile("model/model_example2.pb","wb") as f:
        f.write(output_graph_def.SerializeToString())

export_meta_graph函数导出图

tf.train.NewCheckpointReader读取ckpt文件保存的所有变量

tf.train.get_checkpoint_state通过checkpoint文件自动找到目录中最新模型

# 多线程操作

TensorFlow提供FIFOQueue和RandomShuffleQueue两种队列

In [12]:
q = tf.FIFOQueue(2, "int32")
#初始化
init = q.enqueue_many(([0, 10],))
#取出第一个
x = q.dequeue()
y = x+1
#加入队列
q_inc = q.enqueue([y])
with tf.Session() as sess:
    init.run()
    for _ in range(5):
        v, _ = sess.run([x, q_inc])
        print(v)

0
10
1
11
2


tf.Coordinator和tf.QueueRunner完成多线程协同功能

前者主要协同多个进程一起停止，后一个主要用于启动多线程操作同一个队列
使用方法

coord=tf.train.Coordinator()

tf.train.start_queue_runners(sess=sess,coord=coord)

In [ ]:
tf.train.shuffle_batch([example,label],batch_size=batch_size,
                      capacity=capacity, min_after_dequeue=30)
#capacity是队列最大容量 batch_size是batch样例个数

处理不同文件中的样例，使用tf.tain.shuffle_batch_join